In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6937366746985488987
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5717884928
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13611813874916010133
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)
model.evaluate(x_test, y_test)

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2944 - accuracy: 0.9146
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1401 - accuracy: 0.9581
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1047 - accuracy: 0.9685
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0861 - accuracy: 0.9731
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0731 - accuracy: 0.9765
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0633 - accuracy: 0.9794
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0572 - accuracy: 0.9815
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0513 - accuracy: 0.9834
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0480 - accuracy: 0.9838
Epoch 10/10
313/3

[0.06498835980892181, 0.9812999963760376]

In [7]:
tf.debugging.set_log_device_placement(True)

# 텐서 생성
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import time

with tf.device("/gpu:0"):
    x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=np.float32)
    y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)
    W = tf.Variable(tf.random_normal([2,1]), name='weight')
    b = tf.Variable(tf.random_normal([1]), name='bias')

    W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
    b1 = tf.Variable(tf.zeros([2]), name='Bias1')
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)                   

    W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))
    b2 = tf.Variable(tf.zeros([1]), name='Bias2')
    hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

# 여기서부터는 같은 코드를 사용합니다.
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# 시작 시간 측정
start_time = time.time()
start = time.gmtime(start_time)
print("훈련 시작 : %d시 %d분 %d초"%(start.tm_hour, start.tm_min, start.tm_sec))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(20001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
                print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W))  # W로 하나 W2로 하나 차이 없음.

    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

# 끝난 시간 측정
end_time = time.time()
end = time.gmtime(end_time)
print("훈련 끝 : %d시 %d분 %d초"%(end.tm_hour, end.tm_min, end.tm_sec))

# 소요 시간 측정
end_start = end_time - start_time
end_start = time.gmtime(end_start)
print("소요시간 : %d시 %d분 %d초"%(end_start.tm_hour, end_start.tm_min, end_start.tm_sec))

Instructions for updating:
non-resource variables are not supported in the long term
훈련 시작 : 8시 52분 33초
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6

0 0.69501317 [[-0.21063517]
 [ 0.08264329]]
100 0.6941956 [[-0.21063517]
 [ 0.08264329]]
200 0.6940326 [[-0.21063517]
 [ 0.08264329]]
300 0.6939036 [[-0.21063517]
 [ 0.08264329]]
400 0.6937996 [[-0.21063517]
 [ 0.08264329]]
500 0.6937145 [[-0.21063517]
 [ 0.08264329]]
600 0.6936438 [[-0.21063517]
 [ 0.08264329]]
700 0.6935846 [[-0.21063517]
 [ 0.08264329]]
800 0.6935345 [[-0.21063517]
 [ 0.08264329]]
900 0.69349146 [[-0.21063517]
 [ 0.08264329]]
1000 0.6934544 [[-0.21063517]
 [ 0.08264329]]
1100 0.69342196 [[-0.21063517]
 [ 0.08264329]]
1200 0.6933936 [[-0.21063517]
 [ 0.08264329]]
1300 0.69336843 [[-0.21063517]
 [ 0.08264329]]
1400 0.6933459 [[-0.21063517]
 [ 0.08264329]]
1500 0.6933259 [[-0.21063517]
 [ 0.08264329]]
1600 0.6933

16300 0.013877392 [[-0.21063517]
 [ 0.08264329]]
16400 0.01368579 [[-0.21063517]
 [ 0.08264329]]
16500 0.013499322 [[-0.21063517]
 [ 0.08264329]]
16600 0.013317684 [[-0.21063517]
 [ 0.08264329]]
16700 0.013140751 [[-0.21063517]
 [ 0.08264329]]
16800 0.012968337 [[-0.21063517]
 [ 0.08264329]]
16900 0.012800323 [[-0.21063517]
 [ 0.08264329]]
17000 0.01263646 [[-0.21063517]
 [ 0.08264329]]
17100 0.0124766305 [[-0.21063517]
 [ 0.08264329]]
17200 0.012320677 [[-0.21063517]
 [ 0.08264329]]
17300 0.012168478 [[-0.21063517]
 [ 0.08264329]]
17400 0.012019927 [[-0.21063517]
 [ 0.08264329]]
17500 0.011874855 [[-0.21063517]
 [ 0.08264329]]
17600 0.011733156 [[-0.21063517]
 [ 0.08264329]]
17700 0.011594735 [[-0.21063517]
 [ 0.08264329]]
17800 0.011459459 [[-0.21063517]
 [ 0.08264329]]
17900 0.011327203 [[-0.21063517]
 [ 0.08264329]]
18000 0.011197906 [[-0.21063517]
 [ 0.08264329]]
18100 0.011071447 [[-0.21063517]
 [ 0.08264329]]
18200 0.010947794 [[-0.21063517]
 [ 0.08264329]]
18300 0.01082675 [[-0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import time

with tf.device("/CPU:0"):
    x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=np.float32)
    y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)
    W = tf.Variable(tf.random_normal([2,1]), name='weight')
    b = tf.Variable(tf.random_normal([1]), name='bias')

    W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0))
    b1 = tf.Variable(tf.zeros([2]), name='Bias1')
    L2 = tf.sigmoid(tf.matmul(X, W1) + b1)                   

    W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))
    b2 = tf.Variable(tf.zeros([1]), name='Bias2')
    hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

# 여기서부터는 같은 코드를 사용합니다.
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1-Y) * tf.log(1-hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

# 시작 시간 측정
start_time = time.time()
start = time.gmtime(start_time)
print("훈련 시작 : %d시 %d분 %d초"%(start.tm_hour, start.tm_min, start.tm_sec))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(20001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
                print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(W))  # W로 하나 W2로 하나 차이 없음.

    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

# 끝난 시간 측정
end_time = time.time()
end = time.gmtime(end_time)
print("훈련 끝 : %d시 %d분 %d초"%(end.tm_hour, end.tm_min, end.tm_sec))

# 소요 시간 측정
end_start = end_time - start_time
end_start = time.gmtime(end_start)
print("소요시간 : %d시 %d분 %d초"%(end_start.tm_hour, end_start.tm_min, end_start.tm_sec))

훈련 시작 : 8시 53분 26초
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:2b:00.0, compute capability: 8.6

0 0.70074964 [[1.6945169 ]
 [0.18000656]]
100 0.6933495 [[1.6945169 ]
 [0.18000656]]
200 0.69326204 [[1.6945169 ]
 [0.18000656]]
300 0.6931825 [[1.6945169 ]
 [0.18000656]]
400 0.69310707 [[1.6945169 ]
 [0.18000656]]
500 0.69303393 [[1.6945169 ]
 [0.18000656]]
600 0.69296134 [[1.6945169 ]
 [0.18000656]]
700 0.6928878 [[1.6945169 ]
 [0.18000656]]
800 0.69281167 [[1.6945169 ]
 [0.18000656]]
900 0.6927314 [[1.6945169 ]
 [0.18000656]]
1000 0.6926452 [[1.6945169 ]
 [0.18000656]]
1100 0.6925508 [[1.6945169 ]
 [0.18000656]]
1200 0.692446 [[1.6945169 ]
 [0.18000656]]
1300 0.69232786 [[1.6945169 ]
 [0.18000656]]
1400 0.6921929 [[1.6945169 ]
 [0.18000656]]
1500 0.6920371 [[1.6945169 ]
 [0.18000656]]
1600 0.69185483 [[1.6945169 ]
 [0.18000656]]
1700 0.69163954 [[1.6945169 ]
 [0.18000656]]
1800 0.69138235 [[1.6945169 ]
 [0.18

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # Colab에서만 적용되는 부분입니다 (%tensorflow_version)
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

print("GPU Available: ", tf.test.is_gpu_available())